# Download SNA data 
as per http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3459425/#SD2

Data was chosen using data surveyor (git@bitbucket.org:rxncor/cfg-data-chart.git).
See the [saved search page](../data/data_surveyor_sna.pdf) and saved [SNA json data](../data/sna/SNA_4.0_plant.json)


## House keeping tasks


## import all required dependencies

In [ ]:
# standard imports
import urllib2
import os
import json
import StringIO
import pickle

# dataframe and numerical
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
%matplotlib inline

#scipy
from scipy import stats
from scipy.special import erf
from scipy import sqrt



In [ ]:
from IPython.display import HTML
def addToggle():
    return '''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.'''
HTML(addToggle())


## load dataset and download all data

## write xls to results directory

In [ ]:
# load the sample data from the json file
samples_in="../data/sna/SNA_4.0_plant.json"
results_dir = "../results/sna/"
dataframe_out=results_dir+"dataframes_sna.pkl"
subdir="./"
if not os.path.isfile(samples_in):
    print "You need to include a samples file, generate one using git@bitbucket.org:rxncor/cfg-data-chart.git"
    exit(1)
# get json array metadata from previous search of CFG
with open(os.path.join(subdir, samples_in)) as f:
    datajson = json.load(f)
datajson 


In [ ]:
# list which metadata are available

metadata_tags= datajson[0].keys()
metadata_tags

# HINT: never print datajson[i] once the excel files are loaded. rather use datajson[i].keys() and only print the necessary

In [ ]:
# download xls

for sample in datajson:
    try:
        sample["xlsdata"]= urllib2.urlopen(sample["rawdatalink"][0]).read()
    except Exception as e:
        sample["xlsdata"]=None
        print e

In [ ]:
# check xlsdata exists for samples, did the download go wrong somehow or was the data unavailable.
tmp_check_xls=[]
for sample in datajson:
    if sample["xlsdata"] is not None:
        tmp_check_xls.append("OK")
if len(tmp_check_xls)!=len(datajson):
    print "Missing .xls - check what went wrong"

In [ ]:
# write the sample xls data to results directory

if os.path.isdir(results_dir):
    pass
else:
    os.mkdir(results_dir)


for sample in datajson:
    # prettify the outputname
    excel_out_name = sample["sample"].encode('utf-8')
    excel_out_name_ = excel_out_name.replace('\xc2\xb5', "micro")
    excel_out_name_ = excel_out_name_.replace('/', " per ")
    excel_out_name_ = excel_out_name_.replace(' ', "_")
    excel_out_name_ = excel_out_name_.replace(',', "")
    write_excel = open(results_dir+excel_out_name_+".xls",'wb')
    # write the excel file
    write_excel.write(datajson[0]["xlsdata"])

In [ ]:
# confirm the array version 

total_samples=len(datajson)

# check the array versions 
array_version_stats ={}# knew those from the CFG anyway but do it again
for sample in datajson:
    try:
        array_version_stats[sample["experimentarrayversion"]]+=1
    except:
        array_version_stats[sample["experimentarrayversion"]]=1
key_max, value_max = max(array_version_stats.iteritems(), key=lambda x:x[1])
print key_max, value_max


## load xls into dataframe and add dataframe to sample list 

## write out as a pickle


In [ ]:
# load xls into dataframe and add to sample dict

## Set parameters for this analysis
skiprows=2
columnlist=[0,1,2,3,4,5]
sheetname='CoreH PA Results'


for sample in datajson:
    handle = StringIO.StringIO(sample["xlsdata"])
    xlsx = pd.ExcelFile(handle)
    sn=xlsx.sheet_names
    #if len(sn)>1:
    #    for name in sn:
    #        if 'Cancer' in name or 'Sample' in name or 'Serum' in name:
    #            sheetname=name
    #else:
    #    sheetname=sn[0]
    print "Available sheets:", sn
    sample["dataframe"] = pd.read_excel(xlsx, sheetname, skiprows=skiprows, parse_cols=columnlist)

In [ ]:
## pickle as json breaks on excel and sometimes dataframes
import pickle
pck=open(dataframe_out,'w')
pickle.dump(datajson,pck)
pck.close()